# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 7 2022 10:41AM,250154,10,129904,"Virtue Rx, LLC",Released
1,Nov 7 2022 10:40AM,250153,20,8745081,"Emersa Waterbox, LLC",Released
2,Nov 7 2022 10:32AM,250151,10,0086147061,ISDIN Corporation,Released
3,Nov 7 2022 10:31AM,250150,21,632615,"NBTY Global, Inc.",Released
4,Nov 7 2022 10:30AM,250148,21,632614,"NBTY Global, Inc.",Released
5,Nov 7 2022 10:30AM,250147,21,631720,"NBTY Global, Inc.",Released
6,Nov 7 2022 10:29AM,250146,21,631718,"NBTY Global, Inc.",Released
7,Nov 7 2022 10:16AM,250145,19,MSP215476,Methapharm-G,Released
8,Nov 7 2022 10:16AM,250145,19,MSP215568,"Methapharm, Inc.",Released
9,Nov 7 2022 10:16AM,250145,19,MSP215573,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,250148,Released,1
13,250150,Released,1
14,250151,Released,1
15,250153,Released,1
16,250154,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250148,NaN,1.0
250150,NaN,1.0
250151,NaN,1.0
250153,NaN,1.0
250154,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250118,3.0,3.0
250121,1.0,0.0
250130,0.0,1.0
250131,0.0,1.0
250141,2.0,28.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250118,3,3
250121,1,0
250130,0,1
250131,0,1
250141,2,28


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250118,3,3
1,250121,1,0
2,250130,0,1
3,250131,0,1
4,250141,2,28


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250118,3,3
1,250121,1,
2,250130,,1
3,250131,,1
4,250141,2,28


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 7 2022 10:41AM,250154,10,"Virtue Rx, LLC"
1,Nov 7 2022 10:40AM,250153,20,"Emersa Waterbox, LLC"
2,Nov 7 2022 10:32AM,250151,10,ISDIN Corporation
3,Nov 7 2022 10:31AM,250150,21,"NBTY Global, Inc."
4,Nov 7 2022 10:30AM,250148,21,"NBTY Global, Inc."
5,Nov 7 2022 10:30AM,250147,21,"NBTY Global, Inc."
6,Nov 7 2022 10:29AM,250146,21,"NBTY Global, Inc."
7,Nov 7 2022 10:16AM,250145,19,Methapharm-G
8,Nov 7 2022 10:16AM,250145,19,"Methapharm, Inc."
12,Nov 7 2022 10:11AM,250143,10,"CLINUVEL, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 7 2022 10:41AM,250154,10,"Virtue Rx, LLC",,1
1,Nov 7 2022 10:40AM,250153,20,"Emersa Waterbox, LLC",,1
2,Nov 7 2022 10:32AM,250151,10,ISDIN Corporation,,1
3,Nov 7 2022 10:31AM,250150,21,"NBTY Global, Inc.",,1
4,Nov 7 2022 10:30AM,250148,21,"NBTY Global, Inc.",,1
5,Nov 7 2022 10:30AM,250147,21,"NBTY Global, Inc.",,1
6,Nov 7 2022 10:29AM,250146,21,"NBTY Global, Inc.",,1
7,Nov 7 2022 10:16AM,250145,19,Methapharm-G,,5
8,Nov 7 2022 10:16AM,250145,19,"Methapharm, Inc.",,5
9,Nov 7 2022 10:11AM,250143,10,"CLINUVEL, Inc.",,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 10:41AM,250154,10,"Virtue Rx, LLC",1,
1,Nov 7 2022 10:40AM,250153,20,"Emersa Waterbox, LLC",1,
2,Nov 7 2022 10:32AM,250151,10,ISDIN Corporation,1,
3,Nov 7 2022 10:31AM,250150,21,"NBTY Global, Inc.",1,
4,Nov 7 2022 10:30AM,250148,21,"NBTY Global, Inc.",1,
5,Nov 7 2022 10:30AM,250147,21,"NBTY Global, Inc.",1,
6,Nov 7 2022 10:29AM,250146,21,"NBTY Global, Inc.",1,
7,Nov 7 2022 10:16AM,250145,19,Methapharm-G,5,
8,Nov 7 2022 10:16AM,250145,19,"Methapharm, Inc.",5,
9,Nov 7 2022 10:11AM,250143,10,"CLINUVEL, Inc.",3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 10:41AM,250154,10,"Virtue Rx, LLC",1,
1,Nov 7 2022 10:40AM,250153,20,"Emersa Waterbox, LLC",1,
2,Nov 7 2022 10:32AM,250151,10,ISDIN Corporation,1,
3,Nov 7 2022 10:31AM,250150,21,"NBTY Global, Inc.",1,
4,Nov 7 2022 10:30AM,250148,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 10:41AM,250154,10,"Virtue Rx, LLC",1.0,NaN
1,Nov 7 2022 10:40AM,250153,20,"Emersa Waterbox, LLC",1.0,NaN
2,Nov 7 2022 10:32AM,250151,10,ISDIN Corporation,1.0,NaN
3,Nov 7 2022 10:31AM,250150,21,"NBTY Global, Inc.",1.0,NaN
4,Nov 7 2022 10:30AM,250148,21,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 7 2022 10:41AM,250154,10,"Virtue Rx, LLC",1.0,0.0
1,Nov 7 2022 10:40AM,250153,20,"Emersa Waterbox, LLC",1.0,0.0
2,Nov 7 2022 10:32AM,250151,10,ISDIN Corporation,1.0,0.0
3,Nov 7 2022 10:31AM,250150,21,"NBTY Global, Inc.",1.0,0.0
4,Nov 7 2022 10:30AM,250148,21,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1250731,34.0,2.0
16,500251,1.0,1.0
18,250131,1.0,0.0
19,750408,13.0,3.0
20,250153,1.0,0.0
21,1000591,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1250731,34.0,2.0
1,16,500251,1.0,1.0
2,18,250131,1.0,0.0
3,19,750408,13.0,3.0
4,20,250153,1.0,0.0
5,21,1000591,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,34.0,2.0
1,16,1.0,1.0
2,18,1.0,0.0
3,19,13.0,3.0
4,20,1.0,0.0
5,21,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,34.0
1,16,Released,1.0
2,18,Released,1.0
3,19,Released,13.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20,21
Status,,,,,,
Executing,2.0,1.0,0.0,3.0,0.0,0.0
Released,34.0,1.0,1.0,13.0,1.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20,21
0,Executing,2.0,1.0,0.0,3.0,0.0,0.0
1,Released,34.0,1.0,1.0,13.0,1.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20,21
0,Executing,2.0,1.0,0.0,3.0,0.0,0.0
1,Released,34.0,1.0,1.0,13.0,1.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()